In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Imports

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
import seaborn as sns

# Load data

In [ ]:
data_dir = "../data/raw/"
group1_name = "is20016_by4741"

In [ ]:
filepath1 = data_dir + group1_name
timeseries1_filepath = filepath1 + "_timeseries.csv"
labels1_filepath = filepath1 + "_labels.csv"

In [ ]:
timeseries_df = pd.read_csv(timeseries1_filepath, index_col=[0,1,2])
timeseries_dropna = timeseries_df.dropna()

In [ ]:
labels_df = pd.read_csv(labels1_filepath, index_col=[0,1,2])
labels_df = labels_df == 1
labels_df = labels_df.loc[timeseries_dropna.index]

# Load BioDare2 rhythmicity detection results

In [ ]:
rhythm_dir = "../data/interim/"
rhythm_filepath = rhythm_dir + "BioDare_" + group1_name + "_rhythmicity_copy.csv"
rhythm_df_full = pd.read_csv(rhythm_filepath)

In [ ]:
rhythm_df_full

## Get just what I need

In [ ]:
rhythm_df = rhythm_df_full[['Data Id', 'emp p BH Corrected']]

In [ ]:
rhythm_df

# ROC

Classifications

In [ ]:
q = 0.1
classifications = rhythm_df[["emp p BH Corrected"]].values.ravel() < q

False positives and true positives

In [ ]:
true_labels = labels_df.score.to_list()
predicted_labels = classifications.tolist()

In [ ]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(true_labels, predicted_labels)
true_negative, false_positive, false_negative, true_positive = conf_matrix.ravel()
FPR = false_positive / (false_positive + true_negative)
TPR = true_positive / (true_positive + false_negative)

print(f"False Positive Rate: {FPR}")
print(f"True Positive Rate: {TPR}")

Sweep false discovery rates

In [ ]:
# From BioDARE
#vec = np.array([0.0001, 0.001, 0.005, 0.01, 0.05, 0.1])

# Custom
vec = np.power(10, np.linspace(-8, 0, 100))

In [ ]:
true_labels = labels_df.score.to_list()

FPR_axis = []
TPR_axis = []

for q in vec:
    classifications = rhythm_df[["emp p BH Corrected"]].values.ravel() < q
    predicted_labels = classifications.tolist()
    conf_matrix = confusion_matrix(true_labels, predicted_labels)
    true_negative, false_positive, false_negative, true_positive = conf_matrix.ravel()
    FPR = false_positive / (false_positive + true_negative)
    TPR = true_positive / (true_positive + false_negative)
    FPR_axis.append(FPR)
    TPR_axis.append(TPR)

ROC curve

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
ax.plot(FPR_axis, TPR_axis, marker='o')
ax.plot([0,1],[0,1])
ax.set_xlim((0,1))
ax.set_ylim((0,1))
ax.set_xlabel("False positive rate")
ax.set_ylabel("True positive rate")